In [9]:
import sys
import os

# Add the src directory to the Python path
try:
    current_dir = os.path.dirname(__file__)
except NameError:
    current_dir = os.getcwd()  # Fallback to the current working directory

src_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
sys.path.append(src_dir)

# Now import the module from utils
from src.utils.data_layer import DataLayer

Collecting Data:

rating = 1 for every shiur in database. Can make something more advanced later.

In [ ]:
query = ""
df = DataLayer.execute_query(query)
df['rating'] = 1

Create fastai DataLoaders

In [ ]:
from fastai.collab import *

dls = CollabDataLoaders.from_df(df, user_name='user_id', item_name='item_id', rating_name='rating', bs=64)
dls.show_batch()

Run the model

In [ ]:
from fastai.vision.all import *

learn = collab_learner(dls, n_factors=50, y_range=(0, 1), loss_func=BCEWithLogitsLossFlat())
learn.fit_one_cycle(5, 5e-3)

Save the model

In [ ]:
learn.save("saved_models/user_collab_filtering_v1")

Use model

In [ ]:
from typing import Dict, List
from base import BaseModel



class UserCollabFilteringV1(BaseModel):
    def __init__():
        model = learn.load("../saved_models/user_collab_filtering_v1")

    def get_recommendations(self, user_id: str = None, *args, **kwargs) -> List[int]:
        top_n = 10 if kwargs[top_n] is None else kwargs[top_n]     
        all_item_ids = dls.classes['title'].items
        user_idx = dls.classes['user'].o2i[user_id]
        
        # Create a list of user-item pairs
        user_item_pairs = [(user_idx, dls.classes['title'].o2i[item_id]) for item_id in all_item_ids]
        
        # Create a DataLoader for the user-item pairs
        user_item_dl = dls.test_dl(user_item_pairs)
        
        # Get predictions
        preds, _ = learn.get_preds(dl=user_item_dl)
        
        # Squeeze the predictions to remove extra dimensions
        preds = preds.squeeze()
        
        # Sort the predictions in descending order and get the indices
        sorted_idxs = torch.argsort(preds, descending=True)
        
        # Get the top N item indices
        top_items = [all_item_ids[i] for i in sorted_idxs[:top_n]]
        
        return top_items

    def get_weighted_recommendations(self, user_id: str = None, *args, **kwargs) -> Dict[int, float]:
        return super().get_weighted_recommendations(user_id, *args, **kwargs)
    
    def get_best_shiurim(shiur_num:int = 10): #based on highest bias
        movie_bias = learn.model.movie_bias.squeeze()
        idxs = movie_bias.argsort(descending=True)[:shiur_num]
        return [dls.classes['title'][i] for i in idxs]